TODO
- zgenerirat vse vpisne
- funkcija, ki za dani url (leto in semester) najde predmete, ki jih ima študent na urniku (iz hover texta, po tretji br znacki) in jih shrani v tabelo. ce ni predmetov, naj bo prazen vnos
- to naredimo od leta vpisa do letos
- odstranimo vpisne, ki imajo vse urnik entrye prazne

Na url-ju `https://urnik.fri.uni-lj.si/timetable` dobimo seznam vseh urnikov, ki so na voljo. Za nekatere semestre obstaja več urnikov, a ima le en podatke, ostali pa so prazni. Spišemo kodo, ki nam bo poiskala neprazne urnike.

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

url_base = "https://urnik.fri.uni-lj.si"

url_year = []
valid_timetables = []

driver = webdriver.Chrome()
driver.get(url_base + "/timetable")

soup = BeautifulSoup(driver.page_source, 'html.parser')

timetables_section = soup.find('h1', string='Choose timetable').find_parent('div')

# dobimo linke do vseh urnikov
for a_tag in timetables_section.find_all('a'):
    url_year.append(a_tag.get('href'))

# loopamo cez vse urnike
for year in url_year:
    url = url_base + year

    # gremo na zacetno stran urnika za leto in semester v seznamu
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # najdemo sekcijo s skupinami, poiscemo vse programe
    groups_section = soup.find('h2', string='Groups').find_parent('td')
    group_link = groups_section.find_all('a')

    # ce najdemo podatke o skupinah
    if len(group_link) > 0:
        # poiscemo urnik za dodiplomski program vss in preverimo, ali je prazen
        vss = groups_section.find('a', string='1. letnik, Računalništvo in informatika, prva stopnja: visokošolski strokovni')
        vss_url = url_base + vss.get('href')
        
        driver.get(vss_url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # ce ima urnik vsaj en vnos/predmet, ga dodamo na seznam
        subject_link = soup.find_all(class_="link-subject")
        if subject_link:
            valid_timetables.append(vss_url)


# sortiramo in izpisemo
valid_timetables.sort()
index = 0
for timetable in valid_timetables:
    print(f"{index}. {timetable}")
    index = index + 1

0. https://urnik.fri.uni-lj.si/timetable/fri-2018_2019-zimski-1-6/allocations?group=37842
1. https://urnik.fri.uni-lj.si/timetable/fri-2019_2020-letni-1-9/allocations?group=42918
2. https://urnik.fri.uni-lj.si/timetable/fri-2019_2020-zimski-1-1/allocations?group=40595
3. https://urnik.fri.uni-lj.si/timetable/fri-2020_2021-letni-1-9/allocations?group=45271
4. https://urnik.fri.uni-lj.si/timetable/fri-2020_2021-zimski-1-10/allocations?group=43873
5. https://urnik.fri.uni-lj.si/timetable/fri-2020_2021-zimski-drugi-teden/allocations?group=43873
6. https://urnik.fri.uni-lj.si/timetable/fri-2020_2021-zimski-tretji-teden/allocations?group=43873
7. https://urnik.fri.uni-lj.si/timetable/fri-2021_2022-letni-1-1/allocations?group=48173
8. https://urnik.fri.uni-lj.si/timetable/fri-2021_2022-zimski-1-1/allocations?group=46315
9. https://urnik.fri.uni-lj.si/timetable/fri-2022_2023-letni-1-10/allocations?group=51651
10. https://urnik.fri.uni-lj.si/timetable/fri-2022_2023-zimski-1-1/allocations?group=

Pogledamo sortiran seznam urnikov. Odstranili bomo leto `2018/19` iz analize, saj letni urnik nima podatkov o skupinah. Imamo 4 urnike za leto `2020/21`, ročno odstranimo duplikate, tako kot urnik `test-predavanja`. Odstranimo še duplikat zimskega semestra leta `2023/24`. Ostane nam seznam validnih urnikov med leti `2019/20` in `2024/25`.<br><br>Odstranimo še zadnji del linka, da nam ostane osnovni link do urnika.

In [2]:
valid_timetables.pop(16)
valid_timetables.pop(13)
valid_timetables.pop(6)
valid_timetables.pop(5)
valid_timetables.pop(0)

valid_timetables = [url.rpartition('/')[0] for url in valid_timetables]

index = 0
for timetable in valid_timetables:
    print(f"{index}. {timetable}")
    index = index + 1

0. https://urnik.fri.uni-lj.si/timetable/fri-2019_2020-letni-1-9
1. https://urnik.fri.uni-lj.si/timetable/fri-2019_2020-zimski-1-1
2. https://urnik.fri.uni-lj.si/timetable/fri-2020_2021-letni-1-9
3. https://urnik.fri.uni-lj.si/timetable/fri-2020_2021-zimski-1-10
4. https://urnik.fri.uni-lj.si/timetable/fri-2021_2022-letni-1-1
5. https://urnik.fri.uni-lj.si/timetable/fri-2021_2022-zimski-1-1
6. https://urnik.fri.uni-lj.si/timetable/fri-2022_2023-letni-1-10
7. https://urnik.fri.uni-lj.si/timetable/fri-2022_2023-zimski-1-1
8. https://urnik.fri.uni-lj.si/timetable/fri-2023_2024-letni-1-17
9. https://urnik.fri.uni-lj.si/timetable/fri-2023_2024-zimski-1-12
10. https://urnik.fri.uni-lj.si/timetable/fri-2024_2025-letni
11. https://urnik.fri.uni-lj.si/timetable/fri-2024_2025-zimski


Tako dobljene linke shranimo pretvorimo v DataFrame objekt in jih shranimo v .csv dokument.

In [ ]:
import pandas as pd

timetables = pd.DataFrame(valid_timetables)
timetables.to_csv('timetables.csv', index=False)

print(timetables)

                                                    0
0   https://urnik.fri.uni-lj.si/timetable/fri-2019...
1   https://urnik.fri.uni-lj.si/timetable/fri-2019...
2   https://urnik.fri.uni-lj.si/timetable/fri-2020...
3   https://urnik.fri.uni-lj.si/timetable/fri-2020...
4   https://urnik.fri.uni-lj.si/timetable/fri-2021...
5   https://urnik.fri.uni-lj.si/timetable/fri-2021...
6   https://urnik.fri.uni-lj.si/timetable/fri-2022...
7   https://urnik.fri.uni-lj.si/timetable/fri-2022...
8   https://urnik.fri.uni-lj.si/timetable/fri-2023...
9   https://urnik.fri.uni-lj.si/timetable/fri-2023...
10  https://urnik.fri.uni-lj.si/timetable/fri-2024...
11  https://urnik.fri.uni-lj.si/timetable/fri-2024...
